### tokenizer

In [ ]:
with open("log.txt", "r") as file:
    text = file.read()

In [ ]:
# dic[time.strftime("%Y-%m-%d %H:%M:%S")] = random.randint(1, 100)
dic[time.time()] = random.randint(1, 100)

In [ ]:
sorted(dic.keys())

In [ ]:
dic[123] = 456
print(dic)
print((dic.pop(123)))
dic

### REST API

In [ ]:
import requests
import json
import base64
import cv2
import numpy as np


def send_image_to_backend(
    image_np_array=None,
    model_name="gemma3:12b-it-qat",
    prompt_text="caption this image",
    uri="/api/generate",
):
    """
    将 NumPy 图像数组编码为 Base64 并发送到后端服务器。

    Args:
        image_np_array (np.ndarray): 要发送的图像 NumPy 数组。
        model_name (str): 要使用的模型名称。
        prompt_text (str): 图像描述提示。
        url (str): 后端 API 的 URL。
    """

    if image_np_array is not None:
        # 1. 编码为 JPEG 字节流
        # 这里我们使用默认的JPEG质量，你可以根据需要调整
        ret, buffer = cv2.imencode(
            ".jpg", image_np_array, [cv2.IMWRITE_JPEG_QUALITY, 90]
        )

        if not ret:
            print("错误: 图像编码失败！")
            return
        # 2. 字节流转 Base64 字符串
        image_base64 = base64.b64encode(buffer).decode("utf-8")
        # 3. 构建请求体
        payload = {
            "model": model_name,
            "prompt": prompt_text,
            "images": [
                image_base64
            ],  # 注意这里是一个列表，因为 "images" 字段是一个数组
        }
    else:
        payload = {
            "model": model_name,
            "prompt": prompt_text,
        }

    try:
        # 4. 发送 POST 请求
        # requests.post(): 发送 POST 请求
        # url: 目标URL
        # json: 直接发送JSON数据，requests库会自动将其序列化为JSON字符串并设置Content-Type
        # stream:
        response = requests.post(
            "http://localhost:11434" + uri, json=payload, stream=True
        )
        response.raise_for_status()  # 检查HTTP响应状态码，如果不是2xx，则抛出异常

        print("\n请求成功发送！")

        # 打印服务器的响应内容
        print("服务器响应状态码:", response.status_code)
        print("服务器响应内容:")
        return response

    except requests.exceptions.RequestException as e:
        print(f"发送请求时发生错误: {e}")
        if hasattr(e, "response") and e.response is not None:
            print("服务器响应内容:", e.response.text)

In [ ]:
def split_image(img_array, split_count=2):
    """
    将图像数组分割成多个部分。

    Args:
        img_array (np.ndarray): 输入的图像数组。
        split_count (int): 横向或竖向分割的数量，返回。

    Returns:
        list: 分割后的图像数组列表。
    """
    height, width = img_array.shape[:2]
    split_height = height // split_count
    split_width = width // split_count
    images = []

    for i in range(split_count):
        start_row = i * split_height
        end_row = (i + 1) * split_height if i < split_count - 1 else height
        for i in range(split_count):
            start_col = i * split_width
            end_col = (i + 1) * split_width if i < split_count - 1 else width
            images.append(img_array[start_row:end_row, start_col:end_col])

    return images

In [ ]:
img = capture_screen(scale_factor=1)
imgs = split_image(img, split_count=2)
for i, img in enumerate(imgs):
    play(img)

In [ ]:
# --- 示例用法 ---
"""
camera
"""
# example_image_array = capture_camera(scale_factor=2)
# example_image_array = capture_camera(scale_factor=2, color=cv2.COLOR_BGR2RGB) # do not use this, model use BGR
"""
screen
"""
example_image_array = capture_screen(scale_factor=2)
# example_image_array = sc.capture_screen(scale_factor=2, color=cv2.COLOR_BGR2RGB) # do not use this, model use BGR

# image_array_splits = split_image(example_image_array, split_count=2)

play(example_image_array)

### 调用函数发送图像
# response = send_image_to_backend(example_image_array, prompt_text="what color is the skin of the man in the image, specify the color, do not only tell me it's light or dark?")
# response = send_image_to_backend(example_image_array, model_name="llava:7b", prompt_text="what color is the skin of the man in the image, specify the color, do not only tell me it's light or dark?")
# response = send_image_to_backend(None, model_name="nomic-embed-text", prompt_text="how do I look?", uri="/api/embeddings")
# response = send_image_to_backend(
#     None,
#     model_name="qwen3:8b",
#     prompt_text="你需要提取我的工作，并总结我在哪个时间段(相似工作的时间间隔可以延长)主要干了什么(从核心任务，具体工作，关联工具这三个方面总结)，下面是我的屏幕截图日志: Timestamp: 2025-06-19 19:58:19\nHere's the text I can recognize from the image:\n\n*   **OpenCl Python 镜像**\n*   **设置**\n*   **镜像**\n*   **设置镜像**\n*   **使用镜像**\n*   **镜像名称**\n*   **使用镜像**\n*   **选择镜像**\n*   **镜像**\n*   **选择镜像**\n*   **保存**\n*   **取消**\n\n\n\n\nTimestamp: 2025-06-19 20:00:25\nHere's the text I can recognize:\n\n- **URL:** `https://www.google.com/colab?authuser=1#...`\n- **File Input:**\n    - `main_query`: Text\n    - `attached_file`: Simple File\n    - `language_preference`: English\n- **API Key:** (required) \n- **Submit**\n- **Cancel**\n- **Please remember to provide your API key.**\n\n\n\nI cannot extract further data.\nTimestamp: 2025-06-19 20:02:28\nHere's the text I can recognize from the screen:\n\nmain_query: 用代码生成一个文件\nattached_file: 使用你自己的 API\nlanguage_preference: English\nattached_file: 使用你自己的 API 密钥\nfile: 使用你的 API 密钥\nupload_file: 使用你的 API 密钥\nquery: 使用你的 API 密钥\nTimestamp: 2025-06-19 20:04:30\nHere's the recognizable text from the image:\n\n*   OpenLab Python 镜像\n*   API key\n*   main_query: (填写您的中英文片名，可以重复)\n*   attached_file: (选择您的文件)\n*   language_preference: (选择语言，可选)\n\nThe rest appears to be interface elements and coding-related text.\nTimestamp: 2025-06-19 20:06:32\nHere's the text I can recognize from the image:\n\n*   **Data Analysis**\n*   **Uploaded File:** (something that starts with 043c)\n*   **Language Preference:** English\n*   **Language Preference:** English\n*   **Type:** pdf\n*   **Size:** 1.67MB\n*   **Filename:** (unclear)\n*   **language_preference:** English\n\nTimestamp: 2025-06-19 20:08:35\nHere's the text I can recognize:\n\n*   **main_query:** 请输入你想翻译的中文词语 (Enter the Chinese words you want to translate)\n*   **attached_file:** (Single File)\n*   **language_preference:** 语言选择 (Language selection)\n*   **select:** English\n*   **API_KEY:** (Please provide your API Key)\n*   **model:** Youdao\n\n\n\n\nTimestamp: 2025-06-19 20:10:37\nHere's the text I can recognize:\n\nOpenCl Python 使用方法\n你的 API 密钥\n你的 API 密钥\n上传文件\n转录\n语言偏好: (请选择)\n选择文件\n转录\n\n\nTimestamp: 2025-06-19 20:12:39\nHere's the text I can recognize:\n\n*   API Keys\n*   Get started\n*   Your API key\n*   Enter your API key\n*   Upload a file\n*   Upload a file (Simple File)\n*   Language Preference\n*   English\n*   Spanish\n*   Submit\n*   OpenAI Python Library\n*   Learn more\n*   Terms of Service\n*   Privacy Policy\n\nTimestamp: 2025-06-19 20:14:42\nHere's the text I could recognize:\n\n*   **Title:** Google Gemini\n*   **URL:** https://gemini.google.com/app?hl=zh-Hans\n*   **Text area:**\n    *   请设定以下信息：\n    *   Language Preference: (Select)\n    *   你的 API 密钥 (需提供 YOUR_API_KEY)\n    *   上传文件\n    *   你的API密钥 (请提供 YOUR_API_KEY)\n\nLet me know if you need more information!\nTimestamp: 2025-06-19 20:16:45\nHere's the text I can recognize from the image:\n\n*   开始模式\n*   上传文件\n*   Language preference\n*   选择文件\n*   上传文件\n*   上传中...\n*   请稍候\n*   文件上传成功\n*   正在分析文件\n\n\nTimestamp: 2025-06-19 20:18:47\nHere's the text I can recognize from the screenshot:\n\n*   main_query: (Text)\n*   attached_file: (Simple File)\n*   language_preference: (Select)\n*   language_preference: (Select)\n*   Upload files\n*   Query\n*   Clear\n*   Run query\n*   OpenCV\n\n\n\n\nTimestamp: 2025-06-19 20:20:49\nHere's the text I can recognize from the image:\n\n- Pro\n- Model\n- Uploaded file: [filename]\n- Language preference: Select\n- You are required to provide your API key\n- Open your Python Notebook\n- Learn More\n- File\n- Notebook\n- API key\n- Get started \n- Uploaded File: [filename]\n- Language preference: [selected language]\n- File\n- Notebook\n- API Key \n- Get Started\n- Uploaded file: [filename]\n\nTimestamp: 2025-06-19 20:22:52\nHere's the text I can recognize:\n\nOpenVim\nJupyterLab\nSIMD\nQuick Fix\nGo to Definition\nRename Symbol\n\nAlso:\n语言偏好 (Language Preference)\nEnglish\nEspañol\n你的 API 密钥 (Your API Key)\n上传文件 (Upload File)\n请使用你的 API 密钥。 (Please use your API key.)\nTimestamp: 2025-06-19 20:24:54\nHere's the text I can recognize:\n\n*   **Query:** (提供您的中文化语，用于文件上传)\n*   **transfer_method:** local\n*   **uploaded_file:** [ID=0254312345](文件上传)\n*   **type:** pdf\n*   **language_preference:** English\n\nAdditional parts are not clearly readable.\nTimestamp: 2025-06-19 20:26:56\nHere's the text I can recognize:\n\n*   **Google Gemini**\n*   **OpenCV Python 图像标注**\n*   **main_query:** (用于标记的中文文本)\n*   **attached_file:** (Simple File)\n*   **language_preference:** (选择项) English, 中文\n*   **UPLOADED_FILE_ID:** 123456\n*   **type:** pdf\n*   **language_preference:** 中文\n*   **label:**\n*   **request:**  在 PDF 中提取标注数据\n\n",
# )

### screen recognition
response = send_image_to_backend(example_image_array, model_name="qwen2.5vl:latest", prompt_text="caption this screenshot with as much text as you can recognize on the screen")
# response = send_image_to_backend(example_image_array, model_name="gemma3:12b-it-qat", prompt_text="caption this screenshot")
# response = send_image_to_backend(example_image_array, model_name="llava:7b", prompt_text="print as mush text you can recognize on the screen")
# response = send_image_to_backend(image_array_splits[0], prompt_text="print as mush text as you can recognize on the screen")
# response = send_image_to_backend(example_image_array, model_name="qwen2.5vl:latest", prompt_text="name the main windows open on the screen, tell me the name of the application if you are sure, also tell me the content of the windows, if you can recognize any text, print it as mush as you can")

# 也可以模拟从摄像头捕获并发送
# cap = cv2.VideoCapture(0)
# if cap.isOpened():
#     ret, frame = cap.read()
#     if ret:
#         print("\n正在发送摄像头捕获的帧...")
#         send_image_to_backend(frame, prompt_text="describe this scene from my webcam")
#     cap.release()

In [ ]:
import json


def parse_response(response):
    thinking = False
    for line in response.iter_lines():
        obj = json.loads(line)
        if "<think>" in obj["response"]:
            thinking = True
            print("Thinking...")
        elif thinking and "</think>" in obj["response"]:
            thinking = False
            print("Done thinking.")
        elif not thinking:
            print(obj["response"], end="")
            if obj["done"] == True:
                break

In [ ]:
parse_response(response)

### OpenCV

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython import display
import time

In [ ]:
%matplotlib inline
def play(frame):
    fig, ax = plt.subplots(figsize=(19.2, 10.8)) # img_plot = ax.imshow(np.zeros((row, col, channel), dtype=np.uint8)) # 初始化一个黑色图像
    img_plot = ax.imshow(frame) # 初始化一个黑色图像
    # ax.set_title("camera (Jupyter) - press stop button to interrupt")
    ax.axis('off')
    display.display(fig, display_id=True)
    plt.close(fig)

In [ ]:
# channel for gemma is BGR, for matplotlib and cv2.imencode() is RGB
def capture_camera(scale_factor=1, color=None):
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("错误: 无法打开摄像头。请检查摄像头是否连接正常或是否有权限。")
        return

    # preload to get the shape
    ret, frame = cap.read()
    row, col, channel = frame.shape

    if not ret:
        print("错误: 无法读取帧，可能摄像头已断开连接。")
        return

    # OpenCV 读取的图像是 BGR 格式，Matplotlib 需要 RGB 格式
    # cv2.cvtColor(): 颜色空间转换函数
    # frame: 输入图像
    # cv2.COLOR_BGR2RGB: 指定从 BGR 到 RGB 的转换
    if color is not None:
        frame = cv2.cvtColor(frame, color)
    return cv2.resize(frame, (col // scale_factor, row // scale_factor), cv2.INTER_AREA)

In [ ]:
import mss

def capture_screen(scale_factor=1, color=None):
    with mss.mss() as sct:
        # 获取主显示器的尺寸
        monitor = sct.monitors[0]  # monitors[0]是所有显示器的总和, monitors[1]是主显示器

        while True:
            # 截取屏幕
            # .grab() 返回一个 MSS 的 Image 对象，它包含 .rgb 字节数据和 .size (width, height)
            sct_img = sct.grab(monitor)

            # 将 MSS 图像数据转换为 NumPy 数组
            # mss 捕获的格式是 BGRA (蓝、绿、红、透明度)
            frame = np.array(sct_img)

            # OpenCV 使用 BGR 格式，所以我们可以丢弃 Alpha 通道（透明度）
            # 如果你不需要处理透明度，可以直接切片，这样更快
            # img_bgr = img_np[:, :, :3]
            # 或者使用 OpenCV 的函数进行转换，更具可读性
            frame = cv2.cvtColor(frame, cv2.COLOR_BGRA2BGR)

            # --- 在这里可以加入你的 OpenCV 处理代码 ---
            row, col, channel = frame.shape
            if color is not None:
                frame = cv2.cvtColor(frame, color)
            return cv2.resize(
                frame, (col // scale_factor, row // scale_factor), cv2.INTER_AREA
            )

In [ ]:
# 确保在 Jupyter 中显示 Matplotlib 图形
%matplotlib inline

def capture_and_display_camera_jupyter():
    # 尝试打开默认摄像头
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("错误: 无法打开摄像头。请检查摄像头是否连接正常或是否有权限。")
        return
    
    # preload to get the shape
    ret, frame = cap.read()
    row, col, channel = frame.shape

    # 创建一个 Matplotlib 图形和轴，用于显示图像
    # plt.figure(): 创建一个新的图形
    # figsize=(8, 6): 设置图形的大小
    # plt.imshow(): 在图形中显示图像
    # ax.set_title(): 设置子图标题
    # ax.axis('off'): 关闭坐标轴
    fig, ax = plt.subplots(figsize=(19.2, 10.8))
    # img_plot = ax.imshow(np.zeros((row, col, channel), dtype=np.uint8)) # 初始化一个黑色图像
    img_plot = ax.imshow(cv2.cvtColor(frame, cv2.COLOR_RGBA2BGR)) # 初始化一个黑色图像
    ax.set_title("camera (Jupyter) - press stop button to interrupt")
    ax.axis('off')

    try:
        while True:
            ret, frame = cap.read()

            if not ret:
                print("错误: 无法读取帧，可能摄像头已断开连接。")
                break

            # OpenCV 读取的图像是 BGR 格式，Matplotlib 需要 RGB 格式
            # cv2.cvtColor(): 颜色空间转换函数
            # frame: 输入图像
            # cv2.COLOR_BGR2RGB: 指定从 BGR 到 RGB 的转换
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            row_resize = row // 8
            col_resize = col // 8
            frame_rgb = cv2.resize(frame_rgb, (row_resize, col_resize), cv2.INTER_AREA)

            # 更新 Matplotlib 图像数据
            # set_data(): 更新图像数据
            img_plot.set_data(frame_rgb)

            # 重新绘制图形，实现实时更新
            # display.display(): 在 Jupyter 中显示对象
            # plt.gcf(): 获取当前图形
            # display_id=True: 返回一个显示ID，可以用于更新现有显示
            # clear=True: 清除前一个显示，以避免堆积旧图像
            display.display(fig, display_id=True)
            display.clear_output(wait=True) # 清除上一个输出，等待新的输出

            # 每次循环短暂暂停，避免CPU占用过高
            # time.sleep(0.01): 暂停10毫秒
            time.sleep(0.01) # 控制帧率，避免过快刷新

    except KeyboardInterrupt:
        # 允许通过中断内核（如点击Jupyter的停止按钮）来停止
        print("捕获到键盘中断，停止摄像头捕获。")
    finally:
        # 释放摄像头资源
        cap.release()
        # 关闭 Matplotlib 图形
        plt.close(fig)
        print("摄像头已释放，Jupyter 显示已关闭。")

In [ ]:
capture_and_display_camera_jupyter()